In [27]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/home/beans/bespoke')

from constants import *
from imports import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
for i in range(100):
    paths = glob.glob(f"{BLENDER_MEMBANK_ROOT}/**/obs_per_sec.npy", recursive=True)
    obs_per_sec = np.array([np.load(p)[0] for p in paths])
    print(obs_per_sec)
    time.sleep(60*1)

[]
[]
[12.280024  10.4189205 11.360176  11.3594885 11.050564  13.543543
 11.66776   10.41232   13.444382 ]
[12.280024  10.010933  10.121414  10.4189205 11.360176  11.3594885
 11.050564  17.892477  11.66776    9.026386  10.41232   13.444382 ]
[14.095252 14.922111 17.078566 12.446024 14.374646 14.611466 16.039503
 17.69802  16.30859  16.454288 12.718099 14.354999]
[13.019941  14.742221  14.882701  12.446024  16.845236  12.6004095
 18.430998  17.504396  19.497221  19.363333  16.609406  17.116522 ]
[17.557423 14.742221 16.15626  10.511278 17.879473 16.257132 11.784332
 17.504396 14.117428 14.797562 15.125067 17.248   ]
[17.557423 13.309972 19.13036  16.338053 14.558823 16.257132 18.03987
 15.56094  18.480862 14.797562 19.02606  17.09662 ]
[11.482473 16.007664 19.13036  20.718277 14.558823 15.743567 18.03987
 14.75671  21.151575 14.280066 19.02606  16.533857]
[19.815533  14.14768   12.793508  16.977358  12.102394  14.875952
 13.419354  16.814184  15.868388  13.890819  14.389962  14.3470955]

KeyboardInterrupt: 

In [34]:
set_should_stop(True)

In [29]:
set_lr(3e-4)

In [8]:
import numpy as np

In [9]:
a = np.array([1,2,3,4,1,2,3])

In [11]:
np.where(a==1)[0]

array([0, 4])

In [12]:
max(np.nan, 0)

nan

In [13]:
np.clip(12, 10,11)

11